In [116]:
import pandas as pd 
import numpy as np
import requests
import io
import csv
from path import Path
import psycopg2
#from config import db_password

# Expand cols
pd.set_option('display.max_columns', 80)

CREATE DATASET

In [117]:
# Connction to AWS nba database
connection = psycopg2.connect(
    host = 'nba-db.cpmpsi1pfaz0.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = 'caf3rac3',
    database='postgres'
    )
cursor=connection.cursor()

In [118]:
#read in data from database for All NBA table
nba_unpivoted_sql = 'SELECT * FROM public.allnba_unpivoted'
All_NBA_Unpivoted_df = pd.read_sql(nba_unpivoted_sql, con=connection).drop(['Lg'],axis=1)
#rename the players column
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Value':'PlayerName'})
#create column with the start year
All_NBA_Unpivoted_df['Season_Start_Year'] = All_NBA_Unpivoted_df['Season'].apply(lambda row: row[0:4])
All_NBA_Unpivoted_df['Season_Start_Year'] = pd.to_numeric(All_NBA_Unpivoted_df['Season_Start_Year'])
All_NBA_Unpivoted_df.head(15)

,Season,Tm,Position,PlayerName,Season_Start_Year
0,2019-20,1st,C,Anthony Davis,2019
1,2019-20,1st,F,Giannis Antetokounmpo,2019
2,2019-20,1st,F,LeBron James,2019
3,2019-20,1st,G,James Harden,2019
4,2019-20,1st,G,Luka Dončić,2019
5,2019-20,2nd,C,Nikola Jokić,2019
6,2019-20,2nd,F,Kawhi Leonard,2019
7,2019-20,2nd,F,Pascal Siakam,2019
8,2019-20,2nd,G,Damian Lillard,2019
9,2019-20,2nd,G,Chris Paul,2019


In [119]:

#cleaning up the names with crazy characters
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName'].apply(lambda row: row.replace("ć","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("č","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("ž","z"))

#make dummies for position and team (1st, 2nd, 3rd)
All_NBA_Unpivoted_df = pd.get_dummies(All_NBA_Unpivoted_df,columns=['Position','Tm']).drop(['Season','PlayerName'],axis=1)
#rename the columns (cosmetic)
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_C':'AllNBA_Center'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_F':'AllNBA_Forward'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_G':'AllNBA_Guard'})
All_NBA_Unpivoted_df.head(10)

,Season_Start_Year,PlayerName_Formatted,AllNBA_Center,AllNBA_Forward,AllNBA_Guard,Tm_1st,Tm_2nd,Tm_3rd
0,2019,Anthony Davis,1,0,0,1,0,0
1,2019,Giannis Antetokounmpo,0,1,0,1,0,0
2,2019,LeBron James,0,1,0,1,0,0
3,2019,James Harden,0,0,1,1,0,0
4,2019,Luka Doncic,0,0,1,1,0,0
5,2019,Nikola Jokic,1,0,0,0,1,0
6,2019,Kawhi Leonard,0,1,0,0,1,0
7,2019,Pascal Siakam,0,1,0,0,1,0
8,2019,Damian Lillard,0,0,1,0,1,0
9,2019,Chris Paul,0,0,1,0,1,0


In [120]:
#get list of all of the players that have made an all nba team
AllNBAPlayersList = All_NBA_Unpivoted_df['PlayerName_Formatted'].unique()
AllNBAPlayersList

array(['Anthony Davis', 'Giannis Antetokounmpo', 'LeBron James',
       'James Harden', 'Luka Doncic', 'Nikola Jokic', 'Kawhi Leonard',
       'Pascal Siakam', 'Damian Lillard', 'Chris Paul', 'Rudy Gobert',
       'Jayson Tatum', 'Jimmy Butler', 'Ben Simmons', 'Russell Westbrook',
       'Paul George', 'Stephen Curry', 'Joel Embiid', 'Kevin Durant',
       'Kyrie Irving', 'Blake Griffin', 'Kemba Walker',
       'LaMarcus Aldridge', 'DeMar DeRozan', 'Karl-Anthony Towns',
       'Victor Oladipo', 'Isaiah Thomas', 'DeAndre Jordan',
       'Draymond Green', 'John Wall', 'DeMarcus Cousins',
       'Andre Drummond', 'Klay Thompson', 'Kyle Lowry', 'Marc Gasol',
       'Pau Gasol', 'Tim Duncan', 'Joakim Noah', 'Dwight Howard',
       'Kevin Love', 'Tony Parker', 'Al Jefferson', 'Goran Dragic',
       'Kobe Bryant', 'Carmelo Anthony', 'David Lee', 'Dwyane Wade',
       'Andrew Bynum', 'Tyson Chandler', 'Dirk Nowitzki', 'Rajon Rondo',
       'Derrick Rose', "Amar'e Stoudemire", 'Al Horford', 'Za

In [121]:
#read in data from database for season stats table
seasons_stats_sql = 'SELECT * FROM public.seasons_stats'
season_stats_df = pd.read_sql(seasons_stats_sql, con=connection)
#divid year into season start and end
season_stats_df=season_stats_df.rename(columns = {'Year':'Year_season_end'})
season_stats_df['Year_season_end'] = pd.to_numeric(season_stats_df['Year_season_end'])
season_stats_df['Year_season_start'] = (season_stats_df['Year_season_end'] - 1)
season_stats_df['Player'] = season_stats_df['Player'].apply(lambda row: row.replace("*",""))
season_stats_df.head()


,Year_season_end,Player,Pos,Age,Tm,G,GS,MP,PER,TS_pct,3PAr,FTr,ORB_pct,DRB_pct,TRB_pct,AST_pct,STL_pct,BLK_pct,TOV_pct,USG_pct,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG_pct,3P,3PA,3P_pct,2P,2PA,2P_pct,eFG_pct,FT,FTA,FT_pct,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year_season_start
0,2018,Álex Abrines,SG,24,OKC,75,8,1134.0,9.0,0.567,0.759,0.158,2.5,8.9,5.6,3.4,1.7,0.6,7.4,12.7,1.3,1.0,2.2,0.094,-1.9,0.4,-1.5,0.1,115.0,291.0,0.395,84.0,221.0,0.380,31.0,70.0,0.443,0.540,39.0,46.0,0.848,26.0,88.0,114.0,28.0,38.0,8.0,25.0,124.0,353.0,2017
1,2018,Quincy Acy,PF,27,BRK,70,8,1359.0,8.2,0.525,0.800,0.164,3.1,17.1,10.0,6.0,1.2,1.6,13.3,14.4,-0.1,1.1,1.0,0.036,-2.6,0.1,-2.5,-0.2,130.0,365.0,0.356,102.0,292.0,0.349,28.0,73.0,0.384,0.496,49.0,60.0,0.817,40.0,217.0,257.0,57.0,33.0,29.0,60.0,149.0,411.0,2017
2,2018,Steven Adams,C,24,OKC,76,76,2487.0,20.6,0.630,0.003,0.402,16.6,13.9,15.3,5.5,1.8,2.8,13.3,16.7,6.7,3.0,9.7,0.187,1.7,-0.6,1.1,2.0,448.0,712.0,0.629,0.0,2.0,0.000,448.0,710.0,0.631,0.629,160.0,286.0,0.559,384.0,301.0,685.0,88.0,92.0,78.0,128.0,215.0,1056.0,2017
3,2018,Bam Adebayo,C,20,MIA,69,19,1368.0,15.7,0.570,0.021,0.526,9.7,21.6,15.6,11.0,1.2,2.5,13.6,15.9,2.3,1.9,4.2,0.148,-1.1,0.7,-0.4,0.6,174.0,340.0,0.512,0.0,7.0,0.000,174.0,333.0,0.523,0.512,129.0,179.0,0.721,118.0,263.0,381.0,101.0,32.0,41.0,66.0,138.0,477.0,2017
4,2018,Arron Afflalo,SG,32,ORL,53,3,682.0,5.8,0.516,0.432,0.160,0.6,10.1,5.3,6.2,0.3,1.1,10.8,12.5,-0.1,0.2,0.1,0.009,-3.8,-1.5,-5.4,-0.6,65.0,162.0,0.401,27.0,70.0,0.386,38.0,92.0,0.413,0.485,22.0,26.0,0.846,4.0,62.0,66.0,30.0,4.0,9.0,21.0,56.0,179.0,2017


In [122]:
# get list of every nba player
AllPlayersList = season_stats_df.Player.unique()
AllPlayersList_df = pd.DataFrame(AllPlayersList)
#AllPlayersList_df.to_csv('listofplayers_test.csv')

#check and make sure every all-nba player shows up in the season data dataset
#players who weren't in the league in 2017 will show up here, that's okay!
#I cleaned up a bunch of names in the excel
for player in AllNBAPlayersList:
    if player not in AllPlayersList:   
        print(player)

Luka Doncic


In [123]:
#merge allNBA status into season stats
Season_Stats_ML_DF = pd.merge(season_stats_df,All_NBA_Unpivoted_df,how='left',left_on = ['Player','Year_season_start'], right_on = ['PlayerName_Formatted','Season_Start_Year'])

In [124]:
# get rid of duplicate cols
Season_Stats_ML_DF = Season_Stats_ML_DF.drop(['Season_Start_Year','PlayerName_Formatted'],axis=1)

In [125]:
# fill in zeros for players that arent all nba
Season_Stats_ML_DF['AllNBA_Center'] = Season_Stats_ML_DF['AllNBA_Center'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Forward'] = Season_Stats_ML_DF['AllNBA_Forward'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Guard'] = Season_Stats_ML_DF['AllNBA_Guard'].fillna(int(0))
Season_Stats_ML_DF['Tm_1st'] = Season_Stats_ML_DF['Tm_1st'].fillna(int(0))
Season_Stats_ML_DF['Tm_2nd'] = Season_Stats_ML_DF['Tm_2nd'].fillna(int(0))
Season_Stats_ML_DF['Tm_3rd'] = Season_Stats_ML_DF['Tm_3rd'].fillna(int(0))

In [126]:
#get list of positions
Positions_List = Season_Stats_ML_DF['Pos'].unique()
Positions_List

array(['SG', 'PF', 'C', 'PG', 'SF', 'SF-SG', 'PG-SG', 'PF-SF', 'SG-SF',
       'SG-PF', 'C-PF', 'PF-C', 'SF-PF', 'SG-PG', 'SF-C', 'PG-SF', 'C-SF'],
      dtype=object)

In [127]:
# create functions to determine if player is a guard forward and/or center
def is_forward(pos):
    possible_pos = [ 'SF',  'PF', 'PF-C', 'SG-SF', 'PF-SF', 'SF-SG', 'SF-PF', 'C-PF', 'SG-PF', 'C-SF', 'PG-SF']
    if pos in possible_pos: isF = 1
    else: isF = 0
    return isF

def is_guard(pos):
    possible_pos = ['PG','SG','SG-SF','PG-SG','SG-PG','SF-SG','SG-PF','PG-SF']
    if pos in possible_pos: isG = 1
    else: isG = 0
    return isG

def is_center(pos):
    possible_pos = ['C','PF-C','C-PF','C-SF']
    if pos in possible_pos: isC = 1
    else: isC = 0
    return isC

In [128]:
#Create dummy columns for G, F, C
Season_Stats_ML_DF['is_Guard'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_guard(pos) )
Season_Stats_ML_DF['is_Forward'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_forward(pos) )
Season_Stats_ML_DF['is_Center'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_center(pos) )

In [129]:
#Drop the Pos Column since we have dummy columns
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Pos'],axis=1)

In [130]:
# drop year  and player name
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Year_season_end','Player'],axis=1)
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Year_season_start'],axis=1)

Season_Stats_ML_DF.columns

Index(['Age', 'Tm', 'G', 'GS', 'MP', 'PER', 'TS_pct', '3PAr', 'FTr', 'ORB_pct',
       'DRB_pct', 'TRB_pct', 'AST_pct', 'STL_pct', 'BLK_pct', 'TOV_pct',
       'USG_pct', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'FG', 'FGA', 'FG_pct', '3P', '3PA', '3P_pct', '2P', '2PA', '2P_pct',
       'eFG_pct', 'FT', 'FTA', 'FT_pct', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'AllNBA_Center', 'AllNBA_Forward',
       'AllNBA_Guard', 'Tm_1st', 'Tm_2nd', 'Tm_3rd', 'is_Guard', 'is_Forward',
       'is_Center'],
      dtype='object')

In [131]:
#Creating Per Game Statistics

#Points per Game
Season_Stats_ML_DF["PPG"] = Season_Stats_ML_DF["PTS"]/Season_Stats_ML_DF["G"]
#Assits Per Game
Season_Stats_ML_DF["APG"] = Season_Stats_ML_DF["AST"]/Season_Stats_ML_DF["G"]
#Rebounds Per Game
Season_Stats_ML_DF["RPG"] = Season_Stats_ML_DF["TRB"]/Season_Stats_ML_DF["G"]
#Blocks Per Game
Season_Stats_ML_DF["RPG"] = Season_Stats_ML_DF["BLK"]/Season_Stats_ML_DF["G"]
#STLs Per Game
Season_Stats_ML_DF["SPG"] = Season_Stats_ML_DF["STL"]/Season_Stats_ML_DF["G"]

In [132]:
Season_Stats_ML_DF['3PAr'] = Season_Stats_ML_DF['3PAr'].fillna(int(0))
Season_Stats_ML_DF['3P'] = Season_Stats_ML_DF['3P'].fillna(int(0))
Season_Stats_ML_DF['3PA'] = Season_Stats_ML_DF['3PA'].fillna(int(0))
Season_Stats_ML_DF['3P_pct'] = Season_Stats_ML_DF['3P_pct'].fillna(int(0))


In [133]:
#make dummies for team
#Season_Stats_ML_DF = pd.get_dummies(Season_Stats_ML_DF,columns=['Tm'])
#I don't like this lol

In [134]:
# Fill null values with zero
for col in list(Season_Stats_ML_DF.columns):
    Season_Stats_ML_DF[col] = Season_Stats_ML_DF[col].fillna(int(0))

ALL NBA CENTER MACHINE LEARNING SECTION

In [135]:
#Create Dataset for X & y, make sure it is only centers...we don't need guard or forward data her

# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=78)

X = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Center==1]
X = X.drop(['AllNBA_Center','AllNBA_Forward','AllNBA_Guard','is_Guard','is_Center','is_Forward','WS/48','STL_pct','APG','GS'], axis=1)
X = X.drop(['Tm_1st','Tm_2nd','Tm_3rd'], axis=1)
#I dont think team is that important
X = X.drop(['Tm'], axis=1)
y = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Center==1]
y = y["AllNBA_Center"].values.reshape(-1, 1)

# Splitting into Train and Test sets
# Creating StandardScaler instance
# Fitting Standard Scaller
# Scaling data

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

#oversampling
X_train, y_train = ros.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_Center_Train = X_train

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

center_model = model

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix Centers")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix Centers


,Predicted 0,Predicted 1
Actual 0,1015,6
Actual 1,11,19


Accuracy Score : 0.9838249286393911
Classification Report
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      1021
         1.0       0.76      0.63      0.69        30

    accuracy                           0.98      1051
   macro avg       0.87      0.81      0.84      1051
weighted avg       0.98      0.98      0.98      1051



In [136]:
# We can sort the features by their importance.
model  = model .fit(X_train_scaled, y_train)
importances =model.feature_importances_
center_feature_importance = sorted(zip(model.feature_importances_, X.columns), reverse=True)
center_feature_importance

[(0.752659897768514, 'WS'),
 (0.06061738423040846, 'BLK'),
 (0.06024831249545301, 'TRB'),
 (0.03100544638702937, 'RPG'),
 (0.0285202728044901, 'PER'),
 (0.02223810715829587, 'Age'),
 (0.009965053275406782, 'TOV'),
 (0.007236162118048708, 'DWS'),
 (0.003183150306104743, 'G'),
 (0.0031380863666703638, 'AST'),
 (0.0025510208643595086, 'PF'),
 (0.0019119190543698348, 'TS_pct'),
 (0.0018930316766732301, 'TOV_pct'),
 (0.001886847523009989, 'DRB'),
 (0.0018639313249820405, 'FG_pct'),
 (0.0018359375000000001, 'OWS'),
 (0.001374803241589767, 'OBPM'),
 (0.0012640268650392684, '2P'),
 (0.0012621680020022906, 'DRB_pct'),
 (0.0012584625561562572, 'AST_pct'),
 (0.0006490071614583329, 'BLK_pct'),
 (0.0006484058704453446, 'SPG'),
 (0.0006481609513274336, 'FT'),
 (0.000636456331012424, '3PA'),
 (0.0006351626016260166, '3P'),
 (0.0006346424862779775, '2P_pct'),
 (0.00015555346097312467, 'PPG'),
 (4.744728728044272e-05, 'MP'),
 (3.11423309952774e-05, 'eFG_pct'),
 (0.0, 'VORP'),
 (0.0, 'USG_pct'),
 (0.0, 

ALL NBA GUARD MACHINE LEARNING SECTION

In [137]:
#Create Dataset for X & y, make sure it is only guard...we don't need center or forward data her
ros = RandomOverSampler(random_state=78)

X = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Guard==1]
X = X.drop(['AllNBA_Center','AllNBA_Forward','AllNBA_Guard','is_Guard','is_Center','is_Forward','WS/48','STL_pct','APG','GS'], axis=1)
X = X.drop(['Tm_1st','Tm_2nd','Tm_3rd'], axis=1)
#I dont think team is that important
X = X.drop(['Tm'], axis=1)
y = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Guard==1]
y = y["AllNBA_Guard"].values.reshape(-1, 1)

# Splitting into Train and Test sets
# Creating StandardScaler instance
# Fitting Standard Scaller
# Scaling data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

#oversampling
X_train, y_train = ros.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_Guard_Train = X_train
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier(random_state=78)

model = model.fit(X_train_scaled, y_train)

guard_model = model

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,2031,24
Actual 1,20,32


Accuracy Score : 0.9791172282866635
Classification Report
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      2055
         1.0       0.57      0.62      0.59        52

    accuracy                           0.98      2107
   macro avg       0.78      0.80      0.79      2107
weighted avg       0.98      0.98      0.98      2107



In [138]:
# We can sort the features by their importance.
model  = model .fit(X_train_scaled, y_train)
importances =model.feature_importances_
guard_feature_importance = sorted(zip(model.feature_importances_, X.columns), reverse=True)
guard_feature_importance

[(0.8225661001767586, 'WS'),
 (0.057001895703052995, 'PPG'),
 (0.01289674838606588, 'PER'),
 (0.011662026920933365, 'DRB'),
 (0.011463724243133641, 'DRB_pct'),
 (0.010659929649393229, '3PAr'),
 (0.007994854067775285, 'Age'),
 (0.0054455843816952915, 'USG_pct'),
 (0.005202968141002383, 'OBPM'),
 (0.005122605780690495, 'RPG'),
 (0.004922094292666703, 'AST_pct'),
 (0.004525650132455867, 'TOV'),
 (0.004361062390378351, '3P_pct'),
 (0.0036490277165619183, 'FG_pct'),
 (0.002975033364452392, 'G'),
 (0.0028156433202144093, 'FTA'),
 (0.0027003444001214008, 'FT'),
 (0.002185071925624158, 'VORP'),
 (0.002103046862248289, 'ORB_pct'),
 (0.002084931865304412, 'MP'),
 (0.0019123048700259895, 'FTr'),
 (0.001303833657196691, 'PF'),
 (0.0012824402588458722, 'DBPM'),
 (0.001262809034343684, 'FT_pct'),
 (0.0012572066170322266, 'ORB'),
 (0.0012568479205378544, 'STL'),
 (0.0012567452128040956, 'FG'),
 (0.0012318373115668302, 'FGA'),
 (0.001205719106361378, 'BPM'),
 (0.0009577359375839374, 'BLK'),
 (0.000940

ALL NBA FORWARD MACHINE LEARNING SECTION

In [139]:
#Create Dataset for X & y, make sure it is only guard...we don't need center or forward data her
ros = RandomOverSampler(random_state=78)

X = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Forward==1]
X = X.drop(['AllNBA_Center','AllNBA_Forward','AllNBA_Guard','is_Guard','is_Center','is_Forward','WS/48','STL_pct','APG','GS'], axis=1)
X = X.drop(['Tm_1st','Tm_2nd','Tm_3rd'], axis=1)
#I dont think team is that important
X = X.drop(['Tm'], axis=1)
y = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Forward==1]
y = y["AllNBA_Forward"].values.reshape(-1, 1)

# Splitting into Train and Test sets
# Creating StandardScaler instance
# Fitting Standard Scaller
# Scaling data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

#oversampling
X_train, y_train = ros.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_Forward_Train = X_train
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)
forward_model = model.fit(X_train_scaled, y_train)
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix Forwards")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))


Confusion Matrix Forwards


,Predicted 0,Predicted 1
Actual 0,2039,18
Actual 1,19,35


Accuracy Score : 0.9824727617243013
Classification Report
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      2057
         1.0       0.66      0.65      0.65        54

    accuracy                           0.98      2111
   macro avg       0.83      0.82      0.82      2111
weighted avg       0.98      0.98      0.98      2111



In [140]:
sum(predictions)

53.0

In [141]:
# We can sort the features by their importance.
model  = model .fit(X_train_scaled, y_train)
importances =model.feature_importances_
forward_feature_importance = sorted(zip(model.feature_importances_, X.columns), reverse=True)
forward_feature_importance

[(0.8255849394944266, 'PPG'),
 (0.05712661154494149, 'VORP'),
 (0.029216620145729774, 'PER'),
 (0.011399864935642436, 'USG_pct'),
 (0.010335666455175191, 'TRB_pct'),
 (0.007053485271700184, 'DRB_pct'),
 (0.006601813565050975, 'Age'),
 (0.006350176432718704, 'DWS'),
 (0.00597869722282812, 'eFG_pct'),
 (0.005780609489375142, '3P_pct'),
 (0.005709737744073731, 'WS'),
 (0.004603578903129232, 'FGA'),
 (0.0037385944294725704, '3PA'),
 (0.00245698703497248, 'FT'),
 (0.002097047646152513, 'FT_pct'),
 (0.001907279852832193, 'TOV'),
 (0.0015880527821315606, '2PA'),
 (0.0015816742893747925, 'BLK'),
 (0.001534767329459763, 'TOV_pct'),
 (0.001530609122188225, 'STL'),
 (0.0012595336576632487, 'FG'),
 (0.0009332014863895558, 'FTr'),
 (0.00090062271627812, 'AST_pct'),
 (0.0008935866013071868, 'ORB_pct'),
 (0.0006530049960021912, 'BLK_pct'),
 (0.00064246100243534, 'AST'),
 (0.000637431333847832, 'PTS'),
 (0.0006140092346988978, 'MP'),
 (0.0003303976768780217, 'PF'),
 (0.00032348295004292957, '2P_pct'),

Making Predictions

In [142]:
#filename =  r'C:\Users\jack.getman\OneDrive - WaterBridge Resources, LLC\Desktop\DatacampNew\Bootcamp_Final_Project\2021Data\season_stats2021.csv'
#seasons_stats_2021 = pd.read_csv(filename) 
testing_stats_sql = 'SELECT * FROM public.testing_stats'
seasons_stats_2021 = pd.read_sql(testing_stats_sql, con=connection)
#try:
#    seasons_stats_2021 = seasons_stats_2021.drop(['Unnamed: 0'], axis=1)
#except:
#    print('switch the data source')

#seasons_stats_2021

#Creating Per Game Statistics

#Points per Game
seasons_stats_2021["PPG"] = seasons_stats_2021["PTS"]/seasons_stats_2021["G"]
#Assits Per Game
seasons_stats_2021["APG"] = seasons_stats_2021["AST"]/seasons_stats_2021["G"]
#Rebounds Per Game
seasons_stats_2021["RPG"] = seasons_stats_2021["TRB"]/seasons_stats_2021["G"]
#Blocks Per Game
seasons_stats_2021["RPG"] = seasons_stats_2021["BLK"]/seasons_stats_2021["G"]
#STLs Per Game
seasons_stats_2021["SPG"] = seasons_stats_2021["STL"]/seasons_stats_2021["G"]    
    




In [143]:
filename =  r'Stat_Glossary.xlsx'
stat_glossary = pd.read_excel(filename) 
    
    
scaled_stats = stat_glossary[stat_glossary.Scaled_Per_Game != 0].reset_index().drop(['index'], axis=1).Stat

scaled_seasons_stats_2021 = seasons_stats_2021

def scale_stats(season_df, num_games):
    #scaled_season_df = pd.DataFrame()
    for stat in scaled_stats:
        season_df[stat] = season_df[stat] * 82/num_games
    return season_df

#seasons_stats_2021[seasons_stats_2021.Player == 'LeBron James']

In [144]:
seasons_stats_2021 = scale_stats(seasons_stats_2021,82)
#seasons_stats_2021.head()

In [145]:
seasons_stats_2021[seasons_stats_2021.Player == 'Stephen Curry']

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS_pct,3PAr,FTr,ORB_pct,DRB_pct,TRB_pct,AST_pct,STL_pct,BLK_pct,TOV_pct,USG_pct,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG_pct,3P,3PA,3P_pct,2P,2PA,2P_pct,eFG_pct,FT,FTA,FT_pct,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PPG,APG,RPG,SPG
151,2021,Stephen Curry,PG,32,GSW,71.75,71.75,2450.888889,26.3,0.655,0.587,0.289,1.5,15.4,8.5,30.5,1.7,0.3,12.2,34.8,7.402778,2.847222,10.25,0.201,8.1,0.0,8.1,6.263889,749.388889,1554.583333,0.482,383.805556,912.25,0.421,365.583333,642.333333,0.569,0.605,412.277778,449.861111,0.916,33.027778,359.888889,392.916667,413.416667,87.694444,9.111111,242.583333,135.527778,2294.861111,31.984127,5.761905,0.126984,1.222222


In [146]:
centers_stats_2021 = seasons_stats_2021
forwards_stats_2021 = seasons_stats_2021
guards_stats_2021 = seasons_stats_2021

#seasons_stats_2021=seasons_stats_2021[['Age','G','GS','MP','PER','TS_pct','3PAr','FTr','ORB_pct','DRB_pct','TRB_pct','AST_pct','STL_pct','BLK_pct','TOV_pct','USG_pct','OWS', 'DWS','WS','WS/48','OBPM','DBPM','BPM','VORP','FG','FGA','FG_pct','3P','3PA','3P_pct','2P','2PA','2P_pct','eFG_pct','FT','FTA','FT_pct','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS']]
#seasons_stats_2021

centers_stats_2021['is_Center'] = centers_stats_2021['Pos'].apply(lambda pos: is_center(pos) )
guards_stats_2021['is_Guard'] = guards_stats_2021['Pos'].apply(lambda pos: is_guard(pos) )
forwards_stats_2021['is_Forward'] = forwards_stats_2021['Pos'].apply(lambda pos: is_forward(pos) )

centers_stats_2021=centers_stats_2021[centers_stats_2021['is_Center']==1]
guards_stats_2021=guards_stats_2021[guards_stats_2021['is_Guard']==1]
forwards_stats_2021=forwards_stats_2021[forwards_stats_2021['is_Forward']==1]

centers_stats_2021=centers_stats_2021[centers_stats_2021['is_Center']==1].reset_index()
guards_stats_2021=guards_stats_2021[guards_stats_2021['is_Guard']==1].reset_index()
forwards_stats_2021=forwards_stats_2021[forwards_stats_2021['is_Forward']==1].reset_index()

#centers_stats_2021=centers_stats_2021[['Age','G','MP','PER','TS_pct','3PAr','FTr','ORB_pct','DRB_pct','TRB_pct','AST_pct','BLK_pct','TOV_pct','USG_pct','OWS', 'DWS','WS','OBPM','DBPM','BPM','VORP','FG','FGA','FG_pct','3P','3PA','3P_pct','2P','2PA','2P_pct','eFG_pct','FT','FTA','FT_pct','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','PPG','RPG','SPG']]
#guards_stats_2021=guards_stats_2021[['Age','G','MP','PER','TS_pct','3PAr','FTr','ORB_pct','DRB_pct','TRB_pct','AST_pct','BLK_pct','TOV_pct','USG_pct','OWS', 'DWS','WS','OBPM','DBPM','BPM','VORP','FG','FGA','FG_pct','3P','3PA','3P_pct','2P','2PA','2P_pct','eFG_pct','FT','FTA','FT_pct','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','PPG','RPG','SPG']]
#forwards_stats_2021=forwards_stats_2021[['Age','G','MP','PER','TS_pct','3PAr','FTr','ORB_pct','DRB_pct','TRB_pct','AST_pct','BLK_pct','TOV_pct','USG_pct','OWS', 'DWS','WS','OBPM','DBPM','BPM','VORP','FG','FGA','FG_pct','3P','3PA','3P_pct','2P','2PA','2P_pct','eFG_pct','FT','FTA','FT_pct','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','PPG','RPG','SPG']]

In [147]:
#Predict All- nba Centers
X_scaler = StandardScaler().fit(X_Center_Train)
X_test_scaled = X_scaler.transform(centers_stats_2021[['Age','G','MP','PER','TS_pct','3PAr','FTr','ORB_pct',
                                                       'DRB_pct','TRB_pct','AST_pct','BLK_pct',
                                                       'TOV_pct','USG_pct','OWS', 'DWS','WS','OBPM',
                                                       'DBPM','BPM','VORP','FG','FGA','FG_pct','3P',
                                                       '3PA','3P_pct','2P','2PA','2P_pct','eFG_pct',
                                                       'FT','FTA','FT_pct','ORB','DRB','TRB','AST',
                                                       'STL','BLK','TOV','PF','PTS','PPG','RPG','SPG']])
X_test_scaled = np.nan_to_num(X_test_scaled)
predictions = center_model.predict(X_test_scaled)
predictions_df = pd.DataFrame(predictions)
predictions_df = predictions_df.rename(columns = {predictions_df.columns[0]:'All_NBA_Status'})
centers_stats_2021=centers_stats_2021.join(predictions_df)
centers_stats_2021[centers_stats_2021.All_NBA_Status == 1]

,index,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS_pct,3PAr,FTr,ORB_pct,DRB_pct,TRB_pct,AST_pct,STL_pct,BLK_pct,TOV_pct,USG_pct,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG_pct,3P,3PA,3P_pct,2P,2PA,2P_pct,eFG_pct,FT,FTA,FT_pct,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PPG,APG,RPG,SPG,is_Center,is_Guard,is_Forward,All_NBA_Status
46,189,2021,Joel Embiid,C,26,PHI,58.083333,58.083333,1805.138889,30.3,0.636,0.171,0.61,8.0,29.1,18.7,16.2,1.5,3.9,12.2,35.3,6.377778,3.644444,10.022222,0.266,6.2,1.0,7.2,4.213889,525.027778,1023.861111,0.513,66.055556,175.388889,0.377,458.972222,848.472222,0.541,0.545,536.416667,624.111111,0.859,128.694444,485.166667,613.861111,165.138889,56.944444,78.583333,181.083333,140.083333,1652.527778,28.45098,2.843137,1.352941,0.980392,1,0,0,1.0


In [148]:
#Predict All- nba Guards
X_scaler = StandardScaler().fit(X_Guard_Train)
X_test_scaled = X_scaler.transform(guards_stats_2021[['Age','G','MP','PER','TS_pct','3PAr','FTr','ORB_pct',
                                                      'DRB_pct','TRB_pct','AST_pct','BLK_pct','TOV_pct','USG_pct',
                                                      'OWS', 'DWS','WS','OBPM','DBPM','BPM','VORP','FG','FGA',
                                                      'FG_pct','3P','3PA','3P_pct','2P','2PA','2P_pct','eFG_pct',
                                                      'FT','FTA','FT_pct','ORB','DRB','TRB','AST','STL','BLK',
                                                      'TOV','PF','PTS','PPG','RPG','SPG']])
X_test_scaled = np.nan_to_num(X_test_scaled)
predictions = guard_model.predict(X_test_scaled)
predictions_df = pd.DataFrame(predictions)
predictions_df = predictions_df.rename(columns = {predictions_df.columns[0]:'All_NBA_Status'})
guards_stats_2021=guards_stats_2021.join(predictions_df)
guards_stats_2021[guards_stats_2021.All_NBA_Status == 1]

,index,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS_pct,3PAr,FTr,ORB_pct,DRB_pct,TRB_pct,AST_pct,STL_pct,BLK_pct,TOV_pct,USG_pct,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG_pct,3P,3PA,3P_pct,2P,2PA,2P_pct,eFG_pct,FT,FTA,FT_pct,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PPG,APG,RPG,SPG,is_Center,is_Guard,is_Forward,All_NBA_Status
50,151,2021,Stephen Curry,PG,32,GSW,71.750000,71.750000,2450.888889,26.3,0.655,0.587,0.289,1.5,15.4,8.5,30.5,1.7,0.3,12.2,34.8,7.402778,2.847222,10.250000,0.201,8.1,0.0,8.1,6.263889,749.388889,1554.583333,0.482,383.805556,912.250000,0.421,365.583333,642.333333,0.569,0.605,412.277778,449.861111,0.916,33.027778,359.888889,392.916667,413.416667,87.694444,9.111111,242.583333,135.527778,2294.861111,31.984127,5.761905,0.126984,1.222222,0,1,0,1.0
60,171,2021,Luka Dončić,PG,21,DAL,75.166667,75.166667,2576.166667,25.3,0.587,0.406,0.349,2.7,22.9,12.8,44.1,1.4,1.5,15.3,36.0,5.808333,2.961111,8.769444,0.163,6.1,0.6,6.7,5.694444,736.861111,1538.638889,0.479,218.666667,624.111111,0.350,518.194444,914.527778,0.567,0.550,391.777778,536.416667,0.730,61.500000,538.694444,600.194444,645.750000,72.888889,41.000000,320.027778,173.111111,2084.166667,27.727273,8.590909,0.545455,0.969697,0,1,0,1.0
128,328,2021,LeBron James,PG,36,LAL,51.250000,51.250000,1712.888889,24.2,0.602,0.346,0.310,2.2,23.6,12.9,41.8,1.6,1.5,15.2,31.9,3.416667,2.961111,6.377778,0.179,5.9,1.6,7.5,4.100000,480.611111,937.305556,0.513,118.444444,324.583333,0.365,362.166667,612.722222,0.591,0.576,202.722222,290.416667,0.698,33.027778,361.027778,394.055556,398.611111,54.666667,28.472222,191.333333,79.722222,1282.388889,25.022222,7.777778,0.555556,1.066667,0,1,0,1.0
287,675,2021,Russell Westbrook,PG,32,WAS,74.027778,74.027778,2698.027778,19.5,0.509,0.221,0.334,4.9,28.6,16.8,48.6,1.7,0.8,18.0,30.2,0.569444,3.644444,4.213889,0.075,2.5,0.8,3.4,3.644444,619.555556,1409.944444,0.439,97.944444,310.916667,0.315,521.611111,1099.027778,0.475,0.474,308.638889,470.361111,0.656,124.138889,730.027778,854.166667,868.972222,101.361111,26.194444,355.333333,216.388889,1645.694444,22.230769,11.738462,0.353846,1.369231,0,1,0,1.0
297,702,2021,Trae Young,PG,22,ATL,71.750000,71.750000,2420.138889,23.0,0.589,0.357,0.491,2.0,10.5,6.3,45.5,1.2,0.5,16.2,33.0,6.719444,1.480556,8.200000,0.163,5.4,-1.7,3.6,3.416667,554.638889,1266.444444,0.438,154.888889,452.138889,0.343,399.750000,814.305556,0.491,0.499,551.222222,621.833333,0.886,43.277778,235.750000,279.027778,676.500000,60.361111,13.666667,297.250000,126.416667,1815.388889,25.301587,9.428571,0.190476,0.841270,0,1,0,1.0


In [149]:
#Predict All- nba Forwards
X_scaler = StandardScaler().fit(X_Forward_Train)
X_test_scaled = X_scaler.transform(forwards_stats_2021[['Age','G','MP','PER','TS_pct','3PAr','FTr','ORB_pct','DRB_pct',
                                                        'TRB_pct','AST_pct','BLK_pct','TOV_pct','USG_pct','OWS', 'DWS',
                                                        'WS','OBPM','DBPM','BPM','VORP','FG','FGA','FG_pct','3P','3PA',
                                                        '3P_pct','2P','2PA','2P_pct','eFG_pct','FT','FTA','FT_pct','ORB',
                                                        'DRB','TRB','AST','STL','BLK','TOV','PF','PTS','PPG','RPG','SPG']])
X_test_scaled = np.nan_to_num(X_test_scaled)
predictions = forward_model.predict(X_test_scaled)
predictions_df = pd.DataFrame(predictions)
predictions_df = predictions_df.rename(columns = {predictions_df.columns[0]:'All_NBA_Status'})
forwards_stats_2021=forwards_stats_2021.join(predictions_df)
forwards_stats_2021[forwards_stats_2021.All_NBA_Status == 1]

,index,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS_pct,3PAr,FTr,ORB_pct,DRB_pct,TRB_pct,AST_pct,STL_pct,BLK_pct,TOV_pct,USG_pct,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG_pct,3P,3PA,3P_pct,2P,2PA,2P_pct,eFG_pct,FT,FTA,FT_pct,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PPG,APG,RPG,SPG,is_Center,is_Guard,is_Forward,All_NBA_Status
5,17,2021,Giannis Antetokounmpo,PF,26,MIL,69.472222,69.472222,2292.583333,29.2,0.633,0.201,0.528,5.3,28.9,17.5,28.7,1.7,3.2,13.2,32.5,7.858333,3.758333,11.616667,0.244,6.2,2.6,8.8,6.263889,712.944444,1252.777778,0.569,76.305556,251.694444,0.303,636.638889,1001.083333,0.636,0.600,453.277778,661.694444,0.685,110.472222,653.722222,764.194444,406.583333,82.000000,83.138889,235.750000,191.333333,1955.472222,28.147541,5.852459,1.196721,1.180328,0,0,1,1.0
44,106,2021,Jimmy Butler,SF,31,MIA,59.222222,59.222222,1987.361111,26.5,0.607,0.139,0.565,6.3,17.1,11.8,35.1,3.1,1.1,10.6,26.6,7.516667,3.075000,10.591667,0.255,5.1,2.5,7.5,4.783333,416.833333,838.222222,0.497,28.472222,116.166667,0.245,388.361111,722.055556,0.538,0.514,408.861111,473.777778,0.863,107.055556,301.805556,408.861111,420.250000,123.000000,20.500000,124.138889,80.861111,1271.000000,21.461538,7.096154,0.346154,2.076923,0,0,1,1.0
63,153,2021,Anthony Davis,PF,27,LAL,41.000000,41.000000,1323.388889,22.1,0.556,0.163,0.349,6.0,21.5,13.8,16.4,1.9,4.5,9.5,29.2,1.480556,2.733333,4.213889,0.152,2.9,1.6,4.5,2.163889,342.805556,698.138889,0.491,29.611111,113.888889,0.260,313.194444,584.250000,0.536,0.512,179.944444,243.722222,0.738,70.611111,255.111111,325.722222,125.277778,51.250000,67.194444,84.277778,68.333333,895.166667,21.833333,3.055556,1.638889,1.250000,0,0,1,1.0
158,390,2021,Kawhi Leonard,SF,29,LAC,59.222222,59.222222,2019.250000,26.0,0.622,0.280,0.325,3.6,17.4,10.7,24.9,2.3,1.1,9.2,28.6,7.288889,2.733333,10.022222,0.238,5.7,1.0,6.7,4.441667,529.583333,1034.111111,0.512,115.027778,289.277778,0.398,414.555556,744.833333,0.557,0.568,297.250000,335.972222,0.885,62.638889,321.166667,383.805556,306.361111,92.250000,23.916667,119.583333,93.388889,1471.444444,24.846154,5.173077,0.403846,1.557692,0,0,1,1.0
266,688,2021,Zion Williamson,PF,20,NOP,69.472222,69.472222,2307.388889,27.1,0.649,0.033,0.510,8.9,14.7,11.8,19.7,1.3,1.8,11.6,29.8,8.086111,1.822222,9.908333,0.205,5.8,-0.4,5.4,4.327778,722.055556,1181.027778,0.611,11.388889,38.722222,0.294,710.666667,1142.305556,0.622,0.616,420.250000,602.472222,0.698,190.194444,312.055556,502.250000,257.388889,64.916667,44.416667,190.194444,153.750000,1875.750000,27.000000,3.704918,0.639344,0.934426,0,0,1,1.0


In [150]:
# Make sure that the games played for 2021 is normalized to 82 games
# 72 games in 2021 
# 1998-1999 50 games
# 2011-2021 66 games